In [1]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
dataset=pd.read_csv('CKD.csv')

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True,dtype=int)

In [5]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset['classification_yes']

In [7]:
indep.shape

(399, 27)

In [9]:
dataset["classification_yes"].value_counts()


classification_yes
1    249
0    150
Name: count, dtype: int64

In [17]:
#split into training set and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)
  

In [19]:
#preprocessig the dataset
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)
print(x_train)


[[ 0.48681432  1.79924415 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]
 [ 0.30305399  0.26080096 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]
 [-0.61574769 -0.50842064 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]
 ...
 [-0.43198735  0.26080096 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]
 [-1.35078903  0.26080096 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]
 [-0.06446668 -1.27764223 -0.68384276 ...  0.51639778 -0.52223297
  -0.44519456]]


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 
grid = GridSearchCV(RandomForestClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1')   
# fitting the model for grid search 
grid.fit(x_train, y_train) 


Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
6 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\Lenovo\anaconda3\Lib\site-packa

GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1', verbose=3)

In [25]:
# print best parameter after tuning 
print(grid.best_params_) 


{'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 10}


In [27]:
re=grid.cv_results_
print(re)


{'mean_fit_time': array([0.00164928, 0.00147276, 0.07610092, 0.2858911 , 0.03359213,
       0.30682259, 0.        , 0.00129175, 0.0333343 , 0.30159597,
       0.0341433 , 0.21739821]), 'std_fit_time': array([0.00082464, 0.00081186, 0.00604691, 0.016125  , 0.00229507,
       0.00942417, 0.        , 0.00205416, 0.00230449, 0.00758683,
       0.00347549, 0.02933445]), 'mean_score_time': array([0.        , 0.        , 0.00842357, 0.01530838, 0.00759373,
       0.01546979, 0.        , 0.        , 0.0068028 , 0.01233926,
       0.00588913, 0.01033254]), 'std_score_time': array([0.        , 0.        , 0.        , 0.00404522, 0.00488836,
       0.00242902, 0.        , 0.        , 0.00246294, 0.0025681 ,
       0.00203033, 0.00257953]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                   'entropy', 'entropy', 'entropy', 'entropy', 'entropy',
                   'entropy'],
             mask=[False, False, False, False, False, False, False, Fa

In [29]:
grid_predictions = grid.predict(x_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)
print(cm)





[[51  0]
 [ 2 80]]


In [31]:
# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)
print(clf_report)

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        51
           1       1.00      0.98      0.99        82

    accuracy                           0.98       133
   macro avg       0.98      0.99      0.98       133
weighted avg       0.99      0.98      0.99       133



In [33]:
print(grid.best_params_)

{'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 10}


In [35]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001649,0.000825,0.000000,0.000000,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
1,0.001473,0.000812,0.000000,0.000000,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
2,0.076101,0.006047,0.008424,0.000000,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",1.000000,0.971429,0.952381,0.969697,1.000000,0.978701,0.018623,5
3,0.285891,0.016125,0.015308,0.004045,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.969697,0.957746,0.984615,0.969697,1.000000,0.976351,0.014576,6
4,0.033592,0.002295,0.007594,0.004888,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.985075,0.957746,0.968750,0.969697,0.984615,0.973177,0.010413,7
5,0.306823,0.009424,0.015470,0.002429,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.985075,0.971429,0.984615,0.969697,1.000000,0.982163,0.010984,3
6,0.000000,0.000000,0.000000,0.000000,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
7,0.001292,0.002054,0.000000,0.000000,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
8,0.033334,0.002304,0.006803,0.002463,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",1.000000,0.985075,0.968750,0.969697,1.000000,0.984704,0.013768,1
9,0.301596,0.007587,0.012339,0.002568,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.985075,0.971429,0.952381,0.985075,1.000000,0.978792,0.016003,4


In [37]:

from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'entropy', 'max_features': 'sqrt', 'n_estimators': 10}: 0.9850141736106648


In [45]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9998804399808704